# First-time setup

Clone the project base repo.

In [ ]:
!git clone https://github.com/yecchen/cs245-project-crag

fatal: destination path 'cs245-project-crag' already exists and is not an empty directory.


In [ ]:
%%capture
!cd /content/cs245-project-crag && pip install -r requirements.txt
!pip install --upgrade openai

In [ ]:
!huggingface-cli login --token "..."
!export CUDA_VISIBLE_DEVICES=0

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# CRAG dataset files located under /home/data
#!cd /home/data && bzip2 crag_task_1_dev_v4_release.jsonl
#!ls -lAh /home/data
!ls -lAh /content/cs245-project-crag/data

total 706M
-rw-r--r-- 1 root root 706M Nov 26 09:13 crag_task_1_dev_v4_release.jsonl.bz2
drwxr-xr-x 2 root root 4.0K Nov 26 09:16 .ipynb_checkpoints


In [ ]:
# import starter code
import sys
sys.path.append('/content/cs245-project-crag/course_code')

# Vanilla/RAG baselines

In [ ]:
# Kills existing service
!pkill -f serve
# Use 'nohup' and & operator to start the inference server in the background
# Note: can also try meta-llama/Llama-3.2-3B-Instruct with --max_model_len=2048, except one of the RAG contexts is 2493 so it will eventually fail
!nohup vllm serve unsloth/Llama-3.2-3B-Instruct-bnb-4bit --gpu_memory_utilization=0.95 --tensor_parallel_size=1 --dtype="half" \
  --quantization="bitsandbytes" --load-format="bitsandbytes" --port=8088 --enforce_eager --max_model_len=4096 &
# Give it time to load the model
!sleep 30

nohup: appending output to 'nohup.out'


In [ ]:
!cd /content/cs245-project-crag/course_code && python generate.py \
  --dataset_path "data/crag_task_1_dev_v4_release.jsonl.bz2" \
  --split 1 \
  --model_name "vanilla_baseline" \
  --llm_name "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" \
  --is_server \
  --vllm_server "http://localhost:8088/v1"

True
2024-12-02 02:46:46.940929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 02:46:46.963895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 02:46:46.970651: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 02:46:46.986892: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-02 02:46:48.203048: W tensorflow

In [ ]:
!cd /content/cs245-project-crag/course_code && python generate.py \
  --dataset_path "data/crag_task_1_dev_v4_release.jsonl.bz2" \
  --split 1 \
  --model_name "rag_baseline" \
  --llm_name "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" \
  --is_server \
  --vllm_server "http://localhost:8088/v1"

True
2024-12-02 03:11:20.488214: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 03:11:20.511796: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 03:11:20.518895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 03:11:20.539545: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-02 03:11:21.579120: W tensorflow

In [ ]:
!cd /content/cs245-project-crag/course_code && python evaluate.py \
  --dataset_path "data/crag_task_1_dev_v4_release.jsonl.bz2" \
  --model_name "vanilla_baseline" \
  --llm_name "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" \
  --eval_llm_name "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" \
  --is_server \
  --vllm_server "http://localhost:8088/v1" \
  --max_retries 10

2024-12-02 04:57:36.979261: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 04:57:37.002343: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 04:57:37.009605: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 04:57:37.025798: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-02 04:57:38.255472: W tensorflow/comp

In [ ]:
!cd /content/cs245-project-crag/course_code && python evaluate.py \
  --dataset_path "data/crag_task_1_dev_v4_release.jsonl.bz2" \
  --model_name "rag_baseline" \
  --llm_name "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" \
  --eval_llm_name "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" \
  --is_server \
  --vllm_server "http://localhost:8088/v1" \
  --max_retries 10

2024-12-02 05:07:09.163830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 05:07:09.185749: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 05:07:09.192528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 05:07:09.208906: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-02 05:07:10.341752: W tensorflow/comp

# RetRobust baseline

In [ ]:
# First-time setup: download LoRA
!huggingface-cli download Ori/llama-2-13b-peft-nq-retrobust
# First-time setup: download Q4_K_M quantized Llama-2-13B GGUF for approximate performance, float16 won't fit on our GPU
# NOTE: GGUF does not seem to work with LoRA on vLLM
#!curl -Lo llama-2-13b-chat.Q4_K_M.gguf 'https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_K_M.gguf?download=true'

!git clone https://github.com/scoliono/FastChat
!git clone https://github.com/oriyor/reasoning-on-cots

Fetching 8 files: 100% 8/8 [00:00<00:00, 25516.68it/s]
/root/.cache/huggingface/hub/models--Ori--llama-2-13b-peft-nq-retrobust/snapshots/4160729bf70221a1ba1e35d9c92b4a717830a75a
fatal: destination path 'FastChat' already exists and is not an empty directory.
Cloning into 'reasoning-on-cots'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 195 (delta 65), reused 120 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (195/195), 2.12 MiB | 22.43 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [ ]:
!cd FastChat && pip install -e .

Obtaining file:///content/FastChat
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for fschat (pyproject.toml) ... done
  Created wheel for fschat: filename=fschat-0.2.36-0.editable-py3-none-any.whl size=15011 sha256=b89807c6141efb6d43d0ccadfda22f90b5837058e4aa062c0c548ee3ada71884
  Stored in directory: /tmp/pip-ephem-wheel-cache-dr91pxpz/wheels/0e/7d/3f/6256e4d259fdebc8665545ea33ca3112027cecb15f3a295311
Successfully built fschat
  Attempting uninstall: fschat
    Found existing installation: fschat 0.2.36
    Uninstalling fschat-0.2.36:
      Successfully uninstalled fschat-0.2.36


In [ ]:
# Kills existing service
!pkill -f fastchat

# Use 'nohup' and & operator to start the inference server in the background

#! vllm serve meta-llama/Llama-2-13b-chat-hf --load-format=bitsandbytes --quantization=bitsandbytes \
#  --kv-cache-dtype=fp8
#  --enable-lora --lora-modules='retrobust=/root/.cache/huggingface/hub/models--Ori--llama-2-13b-peft-nq-retrobust/snapshots/4160729bf70221a1ba1e35d9c92b4a717830a75a' \
#  --gpu_memory_utilization=0.95 --tensor_parallel_size=1 --port=8088 --enforce_eager &

# AWQ 4bit quantized model can fit on a 16GB GPU
#!nohup vllm serve TheBloke/Llama-2-13B-chat-GPTQ --tokenizer meta-llama/Llama-2-13b-chat-hf \
#--enable-lora --lora-modules='retrobust=/root/.cache/huggingface/hub/models--Ori--llama-2-13b-peft-nq-retrobust/snapshots/4160729bf70221a1ba1e35d9c92b4a717830a75a' \
#--gpu_memory_utilization=0.95 --tensor_parallel_size=1 --port=8088 --enforce_eager &
# Give it time to load the model
#!sleep 30

!export FASTCHAT_WORKER_API_EMBEDDING_BATCH_SIZE=1
!nohup python3 -m fastchat.serve.controller --host=0.0.0.0 &
!sleep 3
!nohup python3 -m fastchat.serve.model_worker --host=0.0.0.0 --model-name "text-davinci-003" --model-path "meta-llama/Llama-2-13b-hf" --load-8bit &
!sleep 200
!nohup python3 -m fastchat.serve.openai_api_server --host=0.0.0.0 &

nohup: appending output to 'nohup.out'
nohup: appending output to 'nohup.out'
nohup: appending output to 'nohup.out'


In [ ]:
# import starter code
import sys
sys.path.append('/content/reasoning-on-cots')
!pip3 uninstall -y openai
!pip3 install openai==0.28 google-search-results wikipedia

Found existing installation: openai 0.28.0
Uninstalling openai-0.28.0:
  Successfully uninstalled openai-0.28.0
  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
  Preparing metadata (setup.py) ... done
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=1fec672a7b9deb023ee28a04217365ef914a8f56ec252a553c64e78ad209a93a
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.6.0 requires openai>=1.0, but you have openai 0.28.0 which is incompatible.


In [ ]:
!cd /content/cs245-project-crag/course_code && python generate_retrobust.py \
  --dataset_path "data/crag_task_1_dev_v4_release.jsonl.bz2" \
  --split 1 \
  --model_name "retrobust_baseline" \
  --llm_name "meta-llama/Llama-2-13b-hf" \
  --is_server

2024-12-01 08:55:04.038053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-01 08:55:04.060226: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-01 08:55:04.067248: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 08:55:04.083745: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-01 08:55:05.419007: W tensorflow/comp

In [ ]:
!cd /content/cs245-project-crag/course_code && python evaluate.py \
  --dataset_path "data/crag_task_1_dev_v4_release.jsonl.bz2" \
  --model_name "retrobust_baseline" \
  --llm_name "meta-llama/Llama-2-13b-hf"

2024-11-28 21:06:54.587570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 21:06:54.609181: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 21:06:54.615791: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 21:06:54.631869: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-28 21:06:55.756041: W tensorflow/comp

# RetRobust++

In [ ]:
!pip3 uninstall -y openai
!pip3 install openai

Found existing installation: openai 0.28.0
Uninstalling openai-0.28.0:
  Successfully uninstalled openai-0.28.0
  Using cached openai-1.55.3-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.55.3-py3-none-any.whl (389 kB)


In [66]:
# Kills existing service
!pkill -f serve

In [67]:
!cd /content/cs245-project-crag/course_code && python generate_retrobust_plusplus.py \
  --dataset_path "data/crag_task_1_dev_v4_release.jsonl.bz2" \
  --split 1 \
  --model_name "retrobust_plusplus" \
  --llm_name "scoliono/retrobust_plusplus_combined_3b_f16"

False
2024-12-03 02:40:10.787230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 02:40:10.811750: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 02:40:10.818729: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 02:40:10.836830: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 02:40:11.968092: W tensorflo

In [68]:
# Kills existing service
!pkill -f serve
# Use 'nohup' and & operator to start the inference server in the background
#!nohup vllm serve scoliono/retrobust_plusplus_f16 --gpu_memory_utilization=0.95 --tensor_parallel_size=1 --dtype="half" --port=8088 --enforce_eager &
!nohup vllm serve unsloth/Llama-3.2-3B-Instruct-bnb-4bit --gpu_memory_utilization=0.95 --tensor_parallel_size=1 --dtype="half" \
  --quantization="bitsandbytes" --load-format="bitsandbytes" --port=8088 --enforce_eager --max_model_len=4096 &
# Give it time to load the model
!sleep 30

nohup: appending output to 'nohup.out'


In [69]:
!cd /content/cs245-project-crag/course_code && python evaluate.py \
  --dataset_path "data/crag_task_1_dev_v4_release.jsonl.bz2" \
  --model_name "retrobust_plusplus" \
  --llm_name "scoliono/retrobust_plusplus_combined_3b_f16" \
  --eval_llm_name "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" \
  --is_server \
  --vllm_server "http://localhost:8088/v1"

2024-12-03 03:10:58.723496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 03:10:58.746009: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 03:10:58.752792: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 03:10:58.768923: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 03:11:00.016307: W tensorflow/comp